### Import library

In [23]:
import torch

### Getting Started

A tensor represents a (possibly multidimensional) array of numerical values. 
- In the one-dimensional case, i.e., when only one axis is needed for the data, a tensor is called a vector.
- With two axes, a tensor is called a matrix. with `k > 2` axes, we drop the specialized names and just refer to the object as a $k^{th}$ - order tensor.

PyTorch provides a variety of functions for creating new tensors prepopulated with values. For example, by invoking `arange(n)`, we can create a vector of evenly spaced values, starting at 0 (included) and ending at **n** (not included). By default, the interval size is 
. Unless otherwise specified, new tensors are stored in main memory and designated for CPU-based computation.

In [24]:
x = torch.arange(12)
x

tensor([ 0,  1,  2,  3,  4,  5,  6,  7,  8,  9, 10, 11])

In [25]:
y = torch.arange(10, dtype=torch.float32)
y

tensor([0., 1., 2., 3., 4., 5., 6., 7., 8., 9.])

Each of these values is called an **element** of the tensor. The tensor x contains 12 elements. We cant inspect the total number of elements in a tensor via its `numel` method.

In [26]:
print(x.numel())
print("Number of elements of y is: ",y.numel())

12
Number of elements of y is:  10


We can access a tensor’s shape (the length along each axis) by inspecting its shape attribute. Because we are dealing with a vector here, the shape contains just a single element and is identical to the size.

In [27]:
x.shape

torch.Size([12])

We can change the shape of a tensor without altering its size or values, by invoking reshape. For example, we can transform our vector x whose shape is (12,) to a matrix X with shape (3, 4). This new tensor retains all elements but reconfigures them into a matrix. Notice that the elements of our vector are laid out one row at a time and thus x[3] == X[0, 3].

In [28]:
X = x.reshape(3, 4)
X

tensor([[ 0,  1,  2,  3],
        [ 4,  5,  6,  7],
        [ 8,  9, 10, 11]])

In [29]:
X.numel()

12

Note that specifying every shape component to reshape is redundant. Because we already know our tensor’s size, we can work out one component of the shape given the rest. For example, given a tensor of size $n$
 and target shape $(h,w)$, we know that $w = n/h$. To automatically infer one component of the shape, we can place a **-1** for the shape component that should be inferred automatically. In our case, instead of calling x.reshape(3, 4), we could have equivalently called `x.reshape(-1, 4)` or `x.reshape(3, -1)`.

In [14]:
Y = y.reshape(-1,5)
Y

tensor([[0., 1., 2., 3., 4.],
        [5., 6., 7., 8., 9.]])

Practitioners often need to work with tensors initialized to contain all 0s or 1s. We can construct a tensor with all elements set to 0 and a shape of (2, 3, 4) via the zeros function.

In [30]:
torch.zeros(2,3,4)

tensor([[[0., 0., 0., 0.],
         [0., 0., 0., 0.],
         [0., 0., 0., 0.]],

        [[0., 0., 0., 0.],
         [0., 0., 0., 0.],
         [0., 0., 0., 0.]]])

Similarly, we can create a tensor with all 1s by invoking ones.

In [32]:
torch.ones((2, 3, 4))

tensor([[[1., 1., 1., 1.],
         [1., 1., 1., 1.],
         [1., 1., 1., 1.]],

        [[1., 1., 1., 1.],
         [1., 1., 1., 1.],
         [1., 1., 1., 1.]]])

We often wish to sample each element randomly (and independently) from a given probability distribution. For example, the parameters of neural networks are often initialized randomly. The following snippet creates a tensor with elements drawn from a standard Gaussian (normal) distribution with mean 0 and standard deviation 1.

In [33]:
torch.randn(3, 4)

tensor([[ 0.9008, -0.5212,  2.1063,  0.6215],
        [ 0.9853, -0.9529, -0.7557,  1.1458],
        [ 0.6122,  0.8553,  2.1288,  0.9179]])

Finally, we can construct tensors by supplying the exact values for each element by supplying (possibly nested) Python list(s) containing numerical literals. Here, we construct a matrix with a list of lists, where the outermost list corresponds to axis 0, and the inner list corresponds to axis 1.

In [34]:
torch.tensor([
    [1, 2, 3, 4],
    [5, 6, 7, 8],
    [9, 10, 11, 12] 
    ])

tensor([[ 1,  2,  3,  4],
        [ 5,  6,  7,  8],
        [ 9, 10, 11, 12]])

### Indexing and Slicing

As with Python lists, we can access tensor elements by indexing
(starting with 0). To access an element based on its position relative
to the end of the list, we can use negative indexing. Finally, we can
access whole ranges of indices via slicing (e.g., ``X[start:stop]``),
where the returned value includes the first index (``start``) *but not
the last* (``stop``). Finally, when only one index (or slice) is
specified for a  $k^{th}$-order tensor, it is applied along
axis 0. Thus, in the following code, ``[-1]`` selects the last row and
``[1:3]`` selects the second and third rows.

In [35]:
X[-1], X[1:3]

(tensor([ 8,  9, 10, 11]),
 tensor([[ 4,  5,  6,  7],
         [ 8,  9, 10, 11]]))

Beyond reading them, we can also write elements of a matrix by specifying indices.

In [ ]:
X[1,2]

tensor(6)

In [38]:
X[1,2] = 25 #change row 1, col 2 to 25
X

tensor([[ 0,  1,  2,  3],
        [ 4,  5, 25,  7],
        [ 8,  9, 10, 11]])

In [39]:
X[1][2]

tensor(25)

If we want to assign multiple elements the same value, we apply the
indexing on the left-hand side of the assignment operation. For
instance, ``[:2, :]`` accesses the first and second rows, where ``:``
takes all the elements along axis 1 (column). While we discussed
indexing for matrices, this also works for vectors and for tensors of
more than two dimensions.

In [41]:
X[:2, :3] = 12 #change from row 0 to 1, and col 0 to 2, to value 12
X 

tensor([[12, 12, 12,  3],
        [12, 12, 12,  7],
        [ 8,  9, 10, 11]])